## Camshift

In [20]:
import numpy as np
import cv2 as cv
import time

cap = cv.VideoCapture("slow_traffic_small.mp4")
# take first frame of the video
ret,frame = cap.read()
# setup initial location of window
x, y, w, h = 300, 200, 100, 50 # simply hardcoded the values
track_window = (x, y, w, h)
# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = cap.read()
    if ret == True:
        time_start = time.time()
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply camshift to get the new location
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv.polylines(frame,[pts],True, 255,2)

        time_end = time.time()

        cv.putText(
            img2,
            f"one hand infer time : {time_end - time_start} s",
            (10, 50),
            cv.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            thickness=2,
        )


        cv.imshow('img2',img2)
        k = cv.waitKey(1) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()

KeyboardInterrupt: 

: 

## dense optical flow

In [2]:
import numpy as np
import cv2 as cv
import time

rtsp_url = "rtsp://192.168.0.181:8080/video/h264"

#cap = cv.VideoCapture(0)
cap = cv.VideoCapture(rtsp_url)


ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    time_start = time.time()

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    time_end = time.time()

    cv.putText(
        bgr,
        f"one hand infer time : {time_end - time_start} s",
        (10, 50),
        cv.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        thickness=2,
    )
    cv.imshow('frame2', bgr)
    k = cv.waitKey(1) & 0xff


    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next 


## sparse optical flow

In [5]:
import numpy as np
import cv2 as cv



cap = cv.VideoCapture("slow_traffic_small.mp4")

#cap = cv.VideoCapture(args.image)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
print(old_frame.dtype)

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
print(type(p0))
print(p0.shape)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    time_start = time.time()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)

    time_end = time.time()

    cv.putText(
        img,
        f"one hand infer time : {time_end - time_start} s",
        (10, 50),
        cv.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        thickness=2,
    )

    cv.putText(
        img,
        f"{p0.shape}",
        (10, 100),
        cv.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        thickness=2,
    )

    cv.imshow('frame', img)



    
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()


uint8
<class 'numpy.ndarray'>
(17, 1, 2)


In [21]:
print(p0.shape)
print(p0)

(17, 1, 2)
[[[258.846    195.7905  ]]

 [[331.2892   131.18066 ]]

 [[296.8859   131.4761  ]]

 [[409.5801   240.33266 ]]

 [[293.94568  196.54816 ]]

 [[175.8542   188.62744 ]]

 [[169.06326  189.6633  ]]

 [[558.0996   242.89606 ]]

 [[328.96304  110.03695 ]]

 [[349.24744  112.4652  ]]

 [[367.62015  113.30155 ]]

 [[338.48712  115.23155 ]]

 [[157.54398  194.1223  ]]

 [[335.87872  121.711494]]

 [[419.42432  328.283   ]]

 [[340.3235   125.27901 ]]

 [[287.75784  153.02162 ]]]


In [22]:
p0.dtype

dtype('float32')

## nail detection and optical flow

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import time
import copy
import cv2
import cv2_nanodet


nanodet = cv2_nanodet.CV2NanoDetONNX(
    model_path="nanodet_finger_v4_048_sim.onnx",
    input_shape=320,
    class_score_th=0.3,
    nms_th=0.6
)

#cap = cv2.VideoCapture(0)
rtsp_url = "rtsp://192.168.0.181:8080/video/h264"
cap = cv2.VideoCapture(rtsp_url)
#cap = cv2.VideoCapture("nanodet_nail_test_crop.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
ret, frame = cap.read()
bboxes, scores, class_ids = nanodet.inference(frame)

cap.release()

pts = []
for bbox in bboxes:
    print(bbox)
    pt = [(bbox[0] + bbox[2]) /2, (bbox[1] + bbox[3])/ 2]
    pts.append(pt)
pts = np.array(pts, dtype=np.float32)
pts = pts.reshape(-1, 1, 2)
print(pts.shape)
print(pts)


False


In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import time
import cv2
import cv2_nanodet

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

nanodet = cv2_nanodet.CV2NanoDetONNX(
    model_path="nanodet_finger_v4_048_sim.onnx",
    input_shape=320,
    class_score_th=0.3,
    nms_th=0.6
)

#cap = cv2.VideoCapture(0)
rtsp_url = "rtsp://192.168.0.181:8080/video/h264"
cap = cv2.VideoCapture(rtsp_url)
#cap = cv2.VideoCapture("nanodet_nail_test_crop.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
ret, frame = cap.read()
bboxes, scores, class_ids = nanodet.inference(frame)

p0 = []
for bbox in bboxes:
    pt = [(bbox[0] + bbox[2]) /2, (bbox[1] + bbox[3])/ 2]
    p0.append(pt)
p0 = np.array(p0, dtype=np.float32)
p0 = p0.reshape(-1, 1, 2)

old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Create a mask image for drawing purposes
mask = np.zeros_like(frame)

while True:
    ret, frame = cap.read()

    time_start = time.time()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()

        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = frame



    time_end = time.time()


    cv2.putText(
        img,
        f"one hand infer time : {time_end - time_start} s",
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        thickness=2,
    )

    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break

    cv2.imshow('frame', img)
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()


uint8
